In [ ]:
from bs4 import BeautifulSoup
import string
import requests
from selenium import webdriver
import spacy
import os
import pandas as pd
import re
from itertools import islice
import numpy as np
from nltk.tokenize import word_tokenize 
import nltk 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
import csv
nltk.download('stopwords')
nltk.download('punkt')
import heapq 
import csv

# Part_1 -  Data Collection


### 1.1. Get the list of books

### Scrap the links in a page containing list of books

Thanks to this function, for each page that arrive in input as href, we are able to scrap the book's Urls in that. N.B: each page has 100 books, so we have to find 100 links for page

In [ ]:
def scrap_url(href, driver , nlp):
    driver.get(href)       
    #time.sleep(5)
    page_soup = BeautifulSoup(driver.page_source, features="lxml")
    
    links = page_soup.find_all('a' , itemprop="url")
    
    lista_links = []
    
    i = 2
    for link in links:
        link_full = link.get('href')
        if (i % 2) == 0 :
            string1 = 'https://www.goodreads.com/en'
            link_full = string1 + link_full
            lista_links.append(link_full)
        
        i = i+1
        
    
    
    
    return lista_links

### Take the book's links

Thanks to this script we are able to read the web-pages that contain the list of best books on the web-site "https://www.goodreads.com" and, for each page, take the Url of the book we are interested in, saving them in a file called "lista_url.txt". In each page, we will scrap the links of 100 books

**lista_url.txt** : This file will contain 30k rows, each row is the link of a book

In [ ]:
#chromedriver = r"C:\Users\thoma\Desktop\HW3_ADM\chromedriver_win32"
driver = webdriver.Chrome(chromedriver)

for i in range (1, 301) :
    href = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="
    
    stringa2 = i
    stringa2 = str(stringa2)
    href = href + stringa2
    
    nlp = spacy.load('en_core_web_sm')
    
    #Let's use the fuction able to find the urls in the page.
    urls = scrap_url(href,driver,nlp)
    
    with open('lista_url.txt', 'a') as f:
        for item in urls:
            f.write("%s\n" % item)

### 1.2. Crawl books

### Download the HTML pages

With this function we are able to save in a folder "html_folder" all the books as .html page.
It takes the page, as link from "lista_url.txt", and download it savingig as .html file in the setted folder/filepath.
After this step, we have all the book's html-pages downloaded, and we are ready to scrap them.

In [ ]:
#filepath = r'C:\Users\thoma\Desktop\HW3_ADM\html_folder'

#Da lanciare come sta settato ora per prendere nuovi file

with open("lista_url.txt") as file_in:
    
    line_count = 1    # article_i at i-th row of "lista_url.txt"
    
    for link in islice(file_in, 1, 30000):
        
        page = requests.get(link, allow_redirects=True)
        contenuto = page.text
        soup = BeautifulSoup (contenuto , features = 'lxml')
        
        
        if line_count == 30002 :
            break
        string1 = "article_"
        string2 = str(line_count)
        string3 = ".html"
        title = string1 + string2 + string3   
        # We automatically assign to each book the name "article_i.html"
        
        
        
        with open (os.path.join(filepath, title), "w",encoding='utf-8' ) as f2:
            f2.write(str(soup))
        line_count += 1